In [ ]:
import pandas as pd
import glob
import os
import shutil
import numpy as np
import pickle
from auto_tqdm import tqdm
import math

In [ ]:
path="../cont_datasets/extracted/"
number_of_segment_to_ensemble = 3

def roundup(x,number_of_segment_to_ensemble):
     return int(math.ceil(x / number_of_segment_to_ensemble)) * number_of_segment_to_ensemble

Zmazanie aktivít, ktoré majú duplucitné identifikátori - nieje možné určiť spravnosť
aktivít zaznamenavajúcich používanie mapy a aktivity s viacerými dotkymi naraz - črty nevieme extrahovať konzistentne, (future work) 

In [ ]:
users = os.listdir(path)
users = [ user + "/" for user in users]
mapa=[5, 11, 17, 23, 6, 12, 18, 24]
id_counter=0
id_all=0
mistakes =0
user_count=0
for user in tqdm(users):

    seasons = os.listdir(path+user)
    seasons = [ season + "/" for season in seasons]
    
    for season in seasons:
        id_to_delete=[]
        df_activity = pd.read_csv(path+user+season+ "Activity.csv", sep=',', header=None)
        id_all+=len(df_activity[0].unique())


        df_activity = df_activity.drop_duplicates(subset=[0,1,2,3,5,7,8,9], keep='last')
        id_to_delete+=list(df_activity.loc[df_activity[9].isin(mapa)][0].unique()) # map aktivity
        
        id_to_delete = list(set(id_to_delete))

        df_touch = pd.read_csv(path+user+season+ "TouchEvent.csv", sep=',', header=None)
        # id_to_delete+=list(df_touch.loc[df_touch[3] == 2][2].unique()) #multitouch
        # id_to_delete+=list(df_touch.loc[df_touch[10] != 0][2].unique()) #otoceny mobil        
        df_touch = df_touch.loc[df_touch[2].isin(id_to_delete)==False]

        if len(df_touch)<2:
            shutil.rmtree((path+user+season)[:-1]) 
            continue

        df_touch.to_csv(path+user+season + "TouchEvent.csv", encoding='utf-8', index=False, header=False)
        
        df_activity = df_activity.loc[df_activity[0].isin(id_to_delete)==False]
        df_activity.to_csv(path+user+season + "Activity.csv", encoding='utf-8', index=False, header=False)
        
        df_acc = pd.read_csv(path+user+season+ "Accelerometer.csv", sep=',', header=None)
        df_acc = df_acc.loc[df_acc[2].isin(id_to_delete)==False]    
        df_acc.to_csv(path+user+season + "Accelerometer.csv", encoding='utf-8', index=False, header=False)
        
        df_gyro = pd.read_csv(path+user+season+ "Gyroscope.csv", sep=',', header=None)
        df_gyro = df_gyro.loc[df_gyro[2].isin(id_to_delete)==False]    
        df_gyro.to_csv(path+user+season + "Gyroscope.csv", encoding='utf-8', index=False, header=False)


        id_counter+= len(id_to_delete)  
    seasons = os.listdir(path+user)
    if len(seasons)==0:
        shutil.rmtree((path+user)[:-1]) 
        


Skontrolovanie či má každá aktivita záznamy v touch, acc aj gyro.

In [ ]:
users = os.listdir(path)
users = [ user + "/" for user in users]
id_counter=0
id_all=0
mistakes =0
user_count=0
for user in tqdm(users):
    
    seasons = os.listdir(path+user)
    seasons = [ season + "/" for season in seasons]
    
    for season in seasons:
        id_to_delete=[]
        df_activity = pd.read_csv(path+user+season+ "Activity.csv", sep=',', header=None)
        
        df_touch = pd.read_csv(path+user+season+ "TouchEvent.csv", sep=',', header=None)
        df_acc = pd.read_csv(path+user+season+ "Accelerometer.csv", sep=',', header=None)    
        df_gyro = pd.read_csv(path+user+season+ "Gyroscope.csv", sep=',', header=None)    
        
        ids_act=list(df_activity[0].unique())
        ids_touch=list(df_touch[2].unique())
        ids_acc=list(df_acc[2].unique())
        ids_gyro=list(df_gyro[2].unique())

        temps = ids_act.copy()
        for temp in temps:
            if temp in ids_act and temp in ids_touch and temp in ids_acc and temp in ids_gyro:
                ids_act.remove(temp)
                ids_touch.remove(temp)
                ids_acc.remove(temp)
                ids_gyro.remove(temp)
        
        df_activity = df_activity.loc[df_activity[0].isin(ids_act)==False]
        if len(df_activity)==0:
            shutil.rmtree((path+user+season)[:-1]) 
            continue
        df_activity.to_csv(path+user+season + "Activity.csv", encoding='utf-8', index=False, header=False)
        
        df_touch = df_touch.loc[df_touch[2].isin(ids_touch)==False]
        if len(df_touch)<2:
            shutil.rmtree((path+user+season)[:-1]) 
            continue
        df_touch.to_csv(path+user+season + "TouchEvent.csv", encoding='utf-8', index=False, header=False)
        
        df_acc = df_acc.loc[df_acc[2].isin(ids_acc)==False]
        if len(df_acc)<2:
            shutil.rmtree((path+user+season)[:-1]) 
            continue
        df_acc.to_csv(path+user+season + "Accelerometer.csv", encoding='utf-8', index=False, header=False)
        
        df_gyro = df_gyro.loc[df_gyro[2].isin(ids_gyro)==False]
        if len(df_gyro)<2:
            shutil.rmtree((path+user+season)[:-1]) 
            continue
        df_gyro.to_csv(path+user+season + "Gyroscope.csv", encoding='utf-8', index=False, header=False)
    seasons = os.listdir(path+user)
    if len(seasons)==0:
        shutil.rmtree((path+user)[:-1]) 


Skontrolovanie ci ma každá aktvita aspoň dva záznami v každom súbore (minimálne z dvoch vieme extrahovať črty)

In [ ]:
users = os.listdir(path)
users = [ user + "/" for user in users]
id_counter=0
id_all=0
mistakes =0
user_count=0
for user in tqdm(users):
    
    seasons = os.listdir(path+user)
    seasons = [ season + "/" for season in seasons]
    
    for season in seasons:
        id_to_delete=[]
        df_activity = pd.read_csv(path+user+season+ "Activity.csv", sep=',', header=None)
        
        df_touch = pd.read_csv(path+user+season+ "TouchEvent.csv", sep=',', header=None)
        df_acc = pd.read_csv(path+user+season+ "Accelerometer.csv", sep=',', header=None)    
        df_gyro = pd.read_csv(path+user+season+ "Gyroscope.csv", sep=',', header=None)    
        
        ids_act=list(df_activity[0].unique())


        for id_act in ids_act:
            if df_touch[df_touch[2] == id_act].shape[0] < 2:
                print("t"+str(user+season))
            
            if df_acc[df_acc[2] == id_act].shape[0] < 2:
                print("a"+str(user+season))
                
            if df_gyro[df_gyro[2] == id_act].shape[0] < 2:
                print("g"+str(user+season))
        

Pridanie názvu stlpcov a spojenie s identifikátormi aktivty 

In [ ]:
users = os.listdir(path)
users = [ user + "/" for user in users]

user_count=0
for user in tqdm(users):
    
    seasons = os.listdir(path+user)
    seasons = [ season + "/" for season in seasons]
    
    for season in seasons:
        id_to_delete=[]
        df_activity = pd.read_csv(path+user+season+ "Activity.csv", sep=',', header=None)
        df_activity.columns = ['activity_id', 'subject_id','session_number','start_time','end_time','relative_start_time','relative_end_time','gesture_scenario','task_id', "content_id"]
        df_activity = df_activity.loc[:,~df_activity.columns.str.contains('time', case=False)] 
        
        df_touch = pd.read_csv(path+user+season+ "TouchEvent.csv", sep=',', header=None)
        df_touch.columns = ['systime','event_time','activity_id','pointer_count','pointer_id','action_id','x','y','pressure','contact_size','phone_orientation']
        df_touch = pd.merge(df_touch, df_activity, on="activity_id").sort_values('systime')
        df_touch.to_csv(path+user+season + "TouchEvent.csv", encoding='utf-8', index=False)

        df_acc = pd.read_csv(path+user+season+ "Accelerometer.csv", sep=',', header=None) 
        df_acc.columns = ['systime','event_time','activity_id','x','y','z','phone_orientation']
        df_acc = pd.merge(df_acc,df_activity, on="activity_id").sort_values('systime')
        df_acc.to_csv(path+user+season + "Accelerometer.csv", encoding='utf-8', index=False)

        df_gyro = pd.read_csv(path+user+season+ "Gyroscope.csv", sep=',', header=None)    
        df_gyro.columns = ['systime','event_time','activity_id','x','y','z','phone_orientation']
        df_gyro = pd.merge(df_gyro,df_activity, on="activity_id").sort_values('systime')
        df_gyro.to_csv(path+user+season + "Gyroscope.csv", encoding='utf-8', index=False)

        os.remove(path+user+season+ "Activity.csv")
 
        

Skontrolovanie či každá dotyková akcia ma začiatok a koniec - hľadanie párov

In [ ]:
users = os.listdir(path)
users = [ user + "/" for user in users]

user_count=0
for user in tqdm(users):
    
    seasons = os.listdir(path+user)
    seasons = [ season + "/" for season in seasons]
    
    for season in tqdm(seasons, leave=False):
        
        df_touch = pd.read_csv(path+user+season+ "TouchEvent.csv", sep=',')
        df_touch.loc[df_touch['action_id'] == 5, 'action_id'] = 0
        df_touch.loc[df_touch['action_id'] == 6, 'action_id'] = 1
        last_zero=0 
        last=1
        bola_zrada=True
        bude_vypis=False
        while (bola_zrada):
            bola_zrada=False
            last=1
            for index in range(last_zero, len(df_touch)):
                if last == df_touch.iloc[index]['action_id'] or df_touch.iloc[index]['action_id']<0 or df_touch.iloc[index]['action_id']>2 or df_touch.iloc[index]['pointer_count']!=1:
                    
                    print('zrada')
                    print(path+user+season)
                    print("row: "+str(index))
                    bola_zrada=True
                    bude_vypis=True
                    index_of_second_zero=None
                    for index2 in range(index+1, len(df_touch)):
                        if df_touch.iloc[index2]['action_id']==0:
                            index_of_second_zero=index2
                            break
                    break    
                if index ==  len(df_touch)-1 and df_touch.iloc[index]['action_id'] != 1:
                    
                    print("zrada na konci")
                    index_of_second_zero = index+1
                    bude_vypis=True
                    bola_zrada = True
                    break
                if not bola_zrada and df_touch.iloc[index]['action_id'] == 0:
                    last_zero=index
                if df_touch.iloc[index]['action_id'] == 0 or df_touch.iloc[index]['action_id'] == 1:
                    last = df_touch.iloc[index]['action_id']
                    
            if bola_zrada:
                df_touch=df_touch.drop(df_touch.index[last_zero:index_of_second_zero]).reset_index(drop=True)
        if bude_vypis:
            df_touch.to_csv(path+user+season + "TouchEvent.csv", encoding='utf-8', index=False)
            print("Vyriesene")  
            print('-------------------')

In [ ]:
users = os.listdir(path)
users = [ user + "/" for user in users]
for user in tqdm(users):
    
    seasons = os.listdir(path+user)
    seasons = [ season + "/" for season in seasons]
    
    for season in seasons:
    
        df_touch = pd.read_csv(path+user+season+ "TouchEvent.csv", sep=',', header=None)
        if len(df_touch)<2:
            shutil.rmtree((path+user+season)[:-1]) 
            continue
        df_touch.to_csv(path+user+season + "TouchEvent.csv", encoding='utf-8', index=False, header=False)
        
    seasons = os.listdir(path+user)
    if len(seasons)==0:
        shutil.rmtree((path+user)[:-1]) 


Skontrolovanie správnej dlžky timestamp

In [ ]:
users = os.listdir(path)
users = [ user + "/" for user in users]

user_count=0
for user in tqdm(users):
    
    seasons = os.listdir(path+user)
    seasons = [ season + "/" for season in seasons]
    
    for season in seasons:

        
        df_touch = pd.read_csv(path+user+season+ "TouchEvent.csv", sep=',')
        if len(df_touch) != len((df_touch['systime']).astype(str).str.extract('^(\d{13})$', expand=False)):
                print('t')
                print(path+user+season)

        df_acc = pd.read_csv(path+user+season+ "Accelerometer.csv", sep=',') 
        if len(df_acc) != len((df_acc['systime']).astype(str).str.extract('^(\d{13})$', expand=False)):
                print('a')
                print(path+user+season)

        df_gyro = pd.read_csv(path+user+season+ "Gyroscope.csv", sep=',')    
        if len(df_gyro) != len((df_gyro['systime']).astype(str).str.extract('^(\d{13})$', expand=False)):
                print('g')
                print(path+user+season)

Pridanie nových stĺpcov
username 
pattern_id -označuje unikatnu dotykovú akciu -  nastavenie defaultnej hodnoty, ktora ak nebude zmenena zaznamy sa zmazu
segment - stlpec pre zachovanie tvaru dátovej množiny zo vstupnej autentifikácie - nebude vyuźitý 

In [ ]:
users = os.listdir(path)
users = [ user + "/" for user in users]

for user in tqdm(users):
    
    seasons = os.listdir(path+user)
    seasons = [ season + "/" for season in seasons]
    
    for season in seasons:

        
        df_touch = pd.read_csv(path+user+season+ "TouchEvent.csv", sep=',')
        df_touch['username'] = "'"+user[:-1]
        df_touch['pattern_id'] = -1
        df_touch.to_csv(path+user+season + "TouchEvent.csv", encoding='utf-8', index=False)

        df_acc = pd.read_csv(path+user+season+ "Accelerometer.csv", sep=',') 
        df_acc['username'] = "'"+user[:-1]
        df_acc['pattern_id'] = -1
        df_acc.to_csv(path+user+season + "Accelerometer.csv", encoding='utf-8', index=False)

        df_gyro = pd.read_csv(path+user+season+ "Gyroscope.csv", sep=',')    
        df_gyro['username'] = "'"+user[:-1]
        df_gyro['pattern_id'] = -1
        df_gyro.to_csv(path+user+season + "Gyroscope.csv", encoding='utf-8', index=False)


Spárovanie záznamov podľa systime unikátných dotykových akcií - zaznamy bez dotykovej akcie su zmazané

In [ ]:
users = os.listdir(path)
users = [ user + "/" for user in users]

user_count=0
for user in tqdm(users):
    
    seasons = os.listdir(path+user)
    seasons = [ season + "/" for season in seasons]
    patter_id = 0
    for season in tqdm(seasons, leave=False):

        pattern_start_timestamp = None
        pattern_end_timestamp = None

        last_acc_index=0
        last_gyro_index=0
        df_touch = pd.read_csv(path+user+season+ "TouchEvent.csv", sep=',')
        df_acc = pd.read_csv(path+user+season+ "Accelerometer.csv", sep=',') 
        df_gyro = pd.read_csv(path+user+season+ "Gyroscope.csv", sep=',') 

        for i in range(len(df_touch)):
            df_touch.loc[i,'pattern_id'] = patter_id  
            if df_touch.iloc[i]['action_id'] == 0:
                pattern_start_timestamp = df_touch.iloc[i]['systime']
                continue
            if df_touch.iloc[i]['action_id'] == 2:
                continue    
            if df_touch.iloc[i]['action_id'] == 1:
                pattern_end_timestamp = df_touch.iloc[i]['systime']
                
                for a in range(last_acc_index,len(df_acc)):
                    last_acc_index = a
                    if df_acc.iloc[a]['systime'] >= pattern_start_timestamp and df_acc.iloc[a]['systime'] <= pattern_end_timestamp:
                        df_acc.loc[a,'pattern_id'] = patter_id
                    if df_acc.iloc[a]['systime'] > pattern_end_timestamp:
                        break

                for a in range(last_gyro_index,len(df_gyro)):
                    last_gyro_index = a
                    if df_gyro.iloc[a]['systime'] >= pattern_start_timestamp and df_gyro.iloc[a]['systime'] <= pattern_end_timestamp:
                        df_gyro.loc[a,'pattern_id'] = patter_id
                    if df_gyro.iloc[a]['systime'] > pattern_end_timestamp:
                        break

                patter_id += 1
        patter_id = roundup(patter_id, number_of_segment_to_ensemble)
        df_touch.to_csv(path+user+season + "TouchEvent.csv", encoding='utf-8', index=False)
        df_acc = df_acc.loc[df_acc['pattern_id'] != -1]
        df_acc.to_csv(path+user+season + "Accelerometer.csv", encoding='utf-8', index=False)
        df_gyro = df_gyro.loc[df_gyro['pattern_id'] != -1]
        df_gyro.to_csv(path+user+season + "Gyroscope.csv", encoding='utf-8', index=False)


Spojenie záznamov všetkých používateľov do spoločných súborov 

Pre zachovanie tvaru predchádzajúceho datasetu je aktivita čítania premenovaná na scenario_show_complex a aktivita písania scenario_show_simple

In [ ]:
path_to_save = "D:/DP/cont_datasets/merged/"

users = os.listdir(path)
users = [ user + "/" for user in users]
final_df_touch=pd.DataFrame()
final_df_acc=pd.DataFrame()
final_df_gyro=pd.DataFrame()

user_count=0
for user in tqdm(users):
    
    seasons = os.listdir(path+user)
    seasons = [ season + "/" for season in seasons]
    for season in seasons:

        df_touch = pd.read_csv(path+user+season+ "TouchEvent.csv", sep=',')
        final_df_touch = final_df_touch.append(df_touch)
        
        df_acc = pd.read_csv(path+user+season+ "Accelerometer.csv", sep=',')
        final_df_acc = final_df_acc.append(df_acc) 

        df_gyro = pd.read_csv(path+user+season+ "Gyroscope.csv", sep=',')
        final_df_gyro = final_df_gyro.append(df_gyro) 

reading=[1, 7, 13, 19, 2, 8, 14, 20]

final_df_touch['scenario'] = np.where(final_df_touch['task_id'].isin(reading), 'scenario_show_complex', 'scenario_show_simple')
final_df_touch["device"] = final_df_touch["gesture_scenario"].astype(str) + final_df_touch["subject_id"].astype(str)
final_df_touch=final_df_touch.rename({'systime': 'timestamp'}, axis=1)
final_df_touch['segment'] = final_df_touch['pattern_id'] % number_of_segment_to_ensemble
final_df_touch['pattern_id'] //= number_of_segment_to_ensemble 
final_df_touch.to_csv(path_to_save + "touch.csv", encoding='utf-8', index=False)

final_df_acc['scenario'] = np.where(final_df_acc['task_id'].isin(reading), 'scenario_show_complex', 'scenario_show_simple')
final_df_acc["device"] = final_df_acc["gesture_scenario"].astype(str) + final_df_acc["subject_id"].astype(str) 
final_df_acc=final_df_acc.rename({'systime': 'timestamp'}, axis=1)
final_df_acc['segment'] = final_df_acc['pattern_id'] % number_of_segment_to_ensemble
final_df_acc['pattern_id'] //= number_of_segment_to_ensemble 
final_df_acc.to_csv(path_to_save + "linear_accelerometer.csv", encoding='utf-8', index=False)

final_df_gyro['scenario'] = np.where(final_df_gyro['task_id'].isin(reading), 'scenario_show_complex', 'scenario_show_simple')
final_df_gyro["device"] = final_df_gyro["gesture_scenario"].astype(str) + final_df_gyro["subject_id"].astype(str) 
final_df_gyro=final_df_gyro.rename({'systime': 'timestamp'}, axis=1)
final_df_gyro['segment'] = final_df_gyro['pattern_id'] % number_of_segment_to_ensemble
final_df_gyro['pattern_id'] //= number_of_segment_to_ensemble 
final_df_gyro.to_csv(path_to_save + "gyroscope.csv", encoding='utf-8', index=False)


In [ ]:
Pre zachovanie tvaru predchádzajúceho datasetu je aktivita čítania premenovaná na scenario_show_complex a aktivita písania scenario_show_simple

In [ ]:
# path = "D:/DP/cont_datasets/merged/"
# reading=[1, 7, 13, 19, 2, 8, 14, 20] # complex

# df_touch = pd.read_csv(path+ "touch.csv", sep=',')
# df_touch['scenario'] = np.where(df_touch['task_id'].isin(reading), 'scenario_show_complex', 'scenario_show_simple')
# df_touch["device"] = df_touch["gesture_scenario"].astype(str) + df_touch["subject_id"].astype(str)
# df_touch.to_csv(path + "touch.csv", encoding='utf-8', index=False)

# df_acc = pd.read_csv(path+ "accelerometer.csv", sep=',')
# df_acc['scenario'] = np.where(df_acc['task_id'].isin(reading), 'scenario_show_complex', 'scenario_show_simple')
# df_acc["device"] = df_acc["gesture_scenario"].astype(str) + df_acc["subject_id"].astype(str) 
# df_acc.to_csv(path + 'accelerometer.csv', index=False)

# df_gyro = pd.read_csv(path+ "gyroscope.csv", sep=',')
# df_gyro['scenario'] = np.where(df_gyro['task_id'].isin(reading), 'scenario_show_complex', 'scenario_show_simple')
# df_gyro["device"] = df_gyro["gesture_scenario"].astype(str) + df_gyro["subject_id"].astype(str) 
# df_gyro.to_csv(path + "gyroscope.csv", encoding='utf-8', index=False)

zmazanie používatelov s malým množstvom záznamov

In [ ]:

path = "../cont_datasets/merged/"

users_to_delete = [737973,815316]

df_touch = pd.read_csv(path+ "touch.csv", sep=',')
df_touch=df_touch.loc[df_touch['username'].isin(users_to_delete)==False ]
df_touch.to_csv(path + "touch.csv", encoding='utf-8', index=False)

df_acc = pd.read_csv(path+ "accelerometer.csv", sep=',')
df_acc=df_acc.loc[df_acc['username'].isin(users_to_delete)==False ]
df_acc.to_csv(path + "accelerometer.csv", encoding='utf-8', index=False)

df_gyro = pd.read_csv(path+ "gyroscope.csv", sep=',')
df_gyro=df_gyro.loc[df_gyro['username'].isin(users_to_delete)==False ]
df_gyro.to_csv(path + "gyroscope.csv", encoding='utf-8', index=False)